<a href="https://colab.research.google.com/github/CarolRibeiro-S/CarolFirisa/blob/main/desafio_grimorio_mystra_carol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio Backend - O Grande Grimório de Mystra (Estágio)
Notebook com execução sequencial contendo:
- Modelagem e validação de magias (campos condicionais)
- Persistência simulada (fake DB em memória)
- Funções com comportamento de endpoints (status_code + JSON)
- Regra extra: cálculo de dano escalado por  nível do slot
- Testes com pytest e cobertura

#Como rodar
Ambiente de execução  -> Reiniciar e executar tudo

## Sessão 1 - Setup e Infraestrutura Arcaica
Nesta sessão eu preparo o ambiente, modelo as magias com validações e monto a persistência simulada (fake DB), incluindo seed data para testar a listagem imediatamente.

In [1]:
!pip -q install pydantic pytest pytest-cov

In [2]:
from typing import Optional, List, Dict, Any, Literal
from datetime import datetime
import uuid

import pydantic
from pydantic import BaseModel, Field, ValidationError, model_validator

print("Pydantic:", pydantic.__version__)

Pydantic: 2.12.3


In [3]:
def generate_id() -> str:
    return str(uuid.uuid4())

def now_iso() -> str:
    return datetime.utcnow().isoformat()

## Modelagem e validações
Regras Principais:
- Se 'exige_componente_material=True', então 'custo_em_ouro' é obrigatório.
- Magias do tipo 'ataque' exigem 'dano_base'.
- 'progresso_por_slot' só é permitido para magias do tipo 'ataque' (mantém domínio coerente e testável).

In [4]:
School = Literal[
    "Abjuração", "Conjuração", "Divinação", "Encantamento",
    "Evocação", "Ilusão", "Necromancia", "Transmutação"
]

SpellType = Literal["ataque", "defesa", "utilidade"]


class SpellBase(BaseModel):
    nome: str = Field(min_length=2, max_length=60)
    escola: School
    nivel: int = Field(ge=0, le=9)
    tipo: SpellType

    exige_componente_material: bool = False
    custo_em_ouro: Optional[float] = Field(default=None, ge=0)

    dano_base: Optional[int] = Field(default=None, ge=0)
    progressao_por_slot: Optional[Dict[str, int]] = None  # ex: {"4": 6, "5": 12}

    @model_validator(mode="after")
    def validate_business_rules(self):
        # Regra 1: componente material -> custo obrigatório
        if self.exige_componente_material and self.custo_em_ouro is None:
            raise ValueError("custo_em_ouro é obrigatório quando exige_componente_material=True")

        # Regra 2: ataque -> dano_base obrigatório
        if self.tipo == "ataque":
            if self.dano_base is None:
                raise ValueError("dano_base é obrigatório para magias do tipo 'ataque'")
        else:
            if self.dano_base is not None:
                raise ValueError("dano_base só pode ser definido para magias do tipo 'ataque'")
            if self.progressao_por_slot is not None:
                raise ValueError("progressao_por_slot só pode ser definido para magias do tipo 'ataque'")

        # Regra 3: valida progressão
        if self.progressao_por_slot is not None:
            for k, v in self.progressao_por_slot.items():
                if not k.isdigit():
                    raise ValueError("progressao_por_slot deve usar chaves numéricas em string, ex: {'4': 6}")
                if v < 0:
                    raise ValueError("progressao_por_slot não pode ter valores negativos")

        return self


class SpellCreate(SpellBase):
    pass


class SpellUpdate(BaseModel):
    # PATCH: tudo opcional
    nome: Optional[str] = Field(default=None, min_length=2, max_length=60)
    escola: Optional[School] = None
    nivel: Optional[int] = Field(default=None, ge=0, le=9)
    tipo: Optional[SpellType] = None

    exige_componente_material: Optional[bool] = None
    custo_em_ouro: Optional[float] = Field(default=None, ge=0)

    dano_base: Optional[int] = Field(default=None, ge=0)
    progressao_por_slot: Optional[Dict[str, int]] = None


class SpellOut(SpellBase):
    id: str
    created_at: str


# Prova de vida (rápida)
SpellCreate(
    nome="Teste",
    escola="Evocação",
    nivel=3,
    tipo="ataque",
    exige_componente_material=False,
    custo_em_ouro=None,
    dano_base=10,
    progressao_por_slot={"4": 6}
)

SpellCreate(nome='Teste', escola='Evocação', nivel=3, tipo='ataque', exige_componente_material=False, custo_em_ouro=None, dano_base=10, progressao_por_slot={'4': 6})

In [5]:
FAKE_DB: List[Dict[str, Any]] = []

def reset_db() -> None:
    global FAKE_DB
    FAKE_DB = []

def seed_data() -> None:
    reset_db()

    seeds = [
        # Ataque + progressão
        {
            "nome": "Bola de Fogo",
            "escola": "Evocação",
            "nivel": 3,
            "tipo": "ataque",
            "exige_componente_material": False,
            "custo_em_ouro": None,
            "dano_base": 28,
            "progressao_por_slot": {"4": 6, "5": 12, "6": 18},
        },
        # Utilidade + componente + custo
        {
            "nome": "Revivificar",
            "escola": "Necromancia",
            "nivel": 3,
            "tipo": "utilidade",
            "exige_componente_material": True,
            "custo_em_ouro": 300.0,
            "dano_base": None,
            "progressao_por_slot": None,
        },
        # Utilidade pura
        {
            "nome": "Desejo",
            "escola": "Conjuração",
            "nivel": 9,
            "tipo": "utilidade",
            "exige_componente_material": False,
            "custo_em_ouro": None,
            "dano_base": None,
            "progressao_por_slot": None,
        },
    ]

    for payload in seeds:
        obj = SpellCreate(**payload).model_dump()
        obj["id"] = generate_id()
        obj["created_at"] = now_iso()
        FAKE_DB.append(obj)

seed_data()
len(FAKE_DB), [s["nome"] for s in FAKE_DB]


/tmp/ipython-input-2437128044.py:5: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().isoformat()


(3, ['Bola de Fogo', 'Revivificar', 'Desejo'])

## Sessão 2 - A API do Grimório (Lógica de Negócio)
Aqui eu implemento funções que simulam endpoints REST: recebem inputs e retornam 'status_code' e payload JSON. Não há servidor real, foco total em domínio, validação e consistência de retorno.

In [6]:
def ok(data: Any, status_code: int = 200) -> Dict[str, Any]:
    return {"status_code": status_code, "data": data}

def fail(message: str, status_code: int = 400, details: Any = None) -> Dict[str, Any]:
    payload = {"status_code": status_code, "error": {"message": message}}
    if details is not None:
        payload["error"]["details"] = details
    return payload

def _find_index(spell_id: str) -> Optional[int]:
    for i, s in enumerate(FAKE_DB):
        if s["id"] == spell_id:
            return i
    return None


In [7]:
def create_spell(payload: Dict[str, Any]) -> Dict[str, Any]:
    try:
        spell = SpellCreate(**payload).model_dump()
    except (ValidationError, ValueError) as e:
        return fail("Validação falhou", 422, str(e))

    obj = dict(spell)
    obj["id"] = generate_id()
    obj["created_at"] = now_iso()
    FAKE_DB.append(obj)

    return ok(obj, 201)


In [8]:
def list_spells(nome: Optional[str] = None, escola: Optional[str] = None, nivel: Optional[int] = None) -> Dict[str, Any]:
    results = FAKE_DB

    if nome:
        needle = nome.strip().lower()
        results = [s for s in results if needle in s["nome"].lower()]

    if escola:
        results = [s for s in results if s["escola"] == escola]

    if nivel is not None:
        results = [s for s in results if s["nivel"] == nivel]

    return ok({"total": len(results), "items": results}, 200)


In [9]:
def get_spell(spell_id: str) -> Dict[str, Any]:
    idx = _find_index(spell_id)
    if idx is None:
        return fail("Magia não encontrada", 404)
    return ok(FAKE_DB[idx], 200)


In [10]:
def update_spell(spell_id: str, payload: Dict[str, Any]) -> Dict[str, Any]:
    idx = _find_index(spell_id)
    if idx is None:
        return fail("Magia não encontrada", 404)

    try:
        patch = SpellUpdate(**payload).model_dump(exclude_none=True)
    except ValidationError as e:
        return fail("Payload de update inválido", 422, str(e))

    current = FAKE_DB[idx]
    merged = {**current, **patch}

    try:
        validated = SpellCreate(**{k: merged.get(k) for k in SpellCreate.model_fields.keys()}).model_dump()
    except (ValidationError, ValueError) as e:
        return fail("Update resultou em magia inválida", 422, str(e))

    merged_final = dict(validated)
    merged_final["id"] = current["id"]
    merged_final["created_at"] = current["created_at"]

    FAKE_DB[idx] = merged_final
    return ok(merged_final, 200)


In [11]:
def delete_spell(spell_id: str) -> Dict[str, Any]:
    idx = _find_index(spell_id)
    if idx is None:
        return fail("Magia não encontrada", 404)

    FAKE_DB.pop(idx)
    return {"status_code": 204, "data": None}


In [12]:
def calcular_dano_escala(id_magia: str, nivel_slot: int) -> Dict[str, Any]:
    idx = _find_index(id_magia)
    if idx is None:
        return fail("Magia não encontrada", 404)

    spell = FAKE_DB[idx]

    if spell["tipo"] != "ataque":
        return fail("Magia não é de ataque", 400)

    progressao = spell.get("progressao_por_slot")
    if not progressao:
        return fail("Magia não possui progressão", 400)

    if nivel_slot < spell["nivel"]:
        return fail("nivel_slot não pode ser menor que o nível da magia", 400)

    bonus = progressao.get(str(nivel_slot), 0)
    dano_total = spell["dano_base"] + bonus

    return ok({
        "id": spell["id"],
        "nome": spell["nome"],
        "nivel_magia": spell["nivel"],
        "nivel_slot": nivel_slot,
        "dano_base": spell["dano_base"],
        "bonus_slot": bonus,
        "dano_total": dano_total,
    }, 200)


In [21]:
# listar
list_spells()

# pegar id da Bola de Fogo
fireball_id = next(s["id"] for s in FAKE_DB if s["nome"] == "Bola de Fogo")

# dano escalado
calcular_dano_escala(fireball_id, 5)

# create inválida (material sem custo)
create_spell({
    "nome": "Selo Arcano",
    "escola": "Abjuração",
    "nivel": 2,
    "tipo": "utilidade",
    "exige_componente_material": True,
    "custo_em_ouro": None,
    "dano_base": None,
    "progressao_por_slot": None,
})


{'status_code': 422,
 'error': {'message': 'Validação falhou',
  'details': "1 validation error for SpellCreate\n  Value error, custo_em_ouro é obrigatório quando exige_componente_material=True [type=value_error, input_value={'nome': 'Selo Arcano', '...gressao_por_slot': None}, input_type=dict]\n    For further information visit https://errors.pydantic.dev/2.12/v/value_error"}}

## Sessão 3 - QA e Rituais de Teste
Os testes garantem que as rotas de sucesso e os principais casos de rro (validação e bordas) permanecem estáveis. Cada teste reinicia o fake DB com seed para evitar deoendência entre testes.

In [19]:
%%writefile test_grimorio.py
import pytest

@pytest.fixture(autouse=True)
def _reset_before_each_test():
    seed_data()
    yield

def test_create_ok():
    resp = create_spell({
        "nome": "Escudo Arcano",
        "escola": "Abjuração",
        "nivel": 1,
        "tipo": "defesa",
        "exige_componente_material": False,
        "custo_em_ouro": None,
        "dano_base": None,
        "progressao_por_slot": None,
    })
    assert resp["status_code"] == 201

def test_create_material_sem_custo_422():
    resp = create_spell({
        "nome": "Portal de Ouro",
        "escola": "Conjuração",
        "nivel": 4,
        "tipo": "utilidade",
        "exige_componente_material": True,
        "custo_em_ouro": None,
        "dano_base": None,
        "progressao_por_slot": None,
    })
    assert resp["status_code"] == 422

def test_list_filtra_por_escola():
    resp = list_spells(escola="Necromancia")
    assert resp["status_code"] == 200
    assert resp["data"]["total"] == 1

def test_get_inexistente_404():
    resp = get_spell("nao-existe")
    assert resp["status_code"] == 404

def test_update_revalida_regra_material():
    created = create_spell({
        "nome": "Luz",
        "escola": "Evocação",
        "nivel": 0,
        "tipo": "utilidade",
        "exige_componente_material": False,
        "custo_em_ouro": None,
        "dano_base": None,
        "progressao_por_slot": None,
    })
    spell_id = created["data"]["id"]

    resp = update_spell(spell_id, {"exige_componente_material": True})
    assert resp["status_code"] == 422

def test_delete_204_e_depois_404():
    fireball_id = next(s["id"] for s in FAKE_DB if s["nome"] == "Bola de Fogo")
    resp = delete_spell(fireball_id)
    assert resp["status_code"] == 204

    resp2 = get_spell(fireball_id)
    assert resp2["status_code"] == 404

def test_calcular_dano_escala_ok():
    fireball_id = next(s["id"] for s in FAKE_DB if s["nome"] == "Bola de Fogo")
    resp = calcular_dano_escala(fireball_id, 5)
    assert resp["status_code"] == 200
    assert resp["data"]["dano_total"] == 40

def test_calcular_dano_escala_magia_nao_ataque():
    revivificar_id = next(s["id"] for s in FAKE_DB if s["nome"] == "Revivificar")
    resp = calcular_dano_escala(revivificar_id, 5)
    assert resp["status_code"] == 400


Overwriting test_grimorio.py


In [18]:
!pytest -q


EEEEEEEE                                                                 [100%]
==================================== ERRORS ====================================
_______________________ ERROR at setup of test_create_ok _______________________

    @pytest.fixture(autouse=True)
    def _reset_before_each_test():
>       seed_data()
        ^^^^^^^^^
E       NameError: name 'seed_data' is not defined

test_grimorio.py:5: NameError
_____________ ERROR at setup of test_create_material_sem_custo_422 _____________

    @pytest.fixture(autouse=True)
    def _reset_before_each_test():
>       seed_data()
        ^^^^^^^^^
E       NameError: name 'seed_data' is not defined

test_grimorio.py:5: NameError
________________ ERROR at setup of test_list_filtra_por_escola _________________

    @pytest.fixture(autouse=True)
    def _reset_before_each_test():
>       seed_data()
        ^^^^^^^^^
E       NameError: name 'seed_data' is not defined

test_grimorio.py:5: NameError
__________________ ERROR a

In [17]:
!pytest --cov -q


EEEEEEEE                                                                 [100%]
==================================== ERRORS ====================================
_______________________ ERROR at setup of test_create_ok _______________________

    @pytest.fixture(autouse=True)
    def _reset_before_each_test():
>       seed_data()
        ^^^^^^^^^
E       NameError: name 'seed_data' is not defined

test_grimorio.py:5: NameError
_____________ ERROR at setup of test_create_material_sem_custo_422 _____________

    @pytest.fixture(autouse=True)
    def _reset_before_each_test():
>       seed_data()
        ^^^^^^^^^
E       NameError: name 'seed_data' is not defined

test_grimorio.py:5: NameError
________________ ERROR at setup of test_list_filtra_por_escola _________________

    @pytest.fixture(autouse=True)
    def _reset_before_each_test():
>       seed_data()
        ^^^^^^^^^
E       NameError: name 'seed_data' is not defined

test_grimorio.py:5: NameError
__________________ ERROR a

## Apêndice — Relatório de IA (Sessão 4)

### Onde utilizei IA como apoio
- Organização do notebook em sessões e checklist de execução sequencial.
- Revisão das regras condicionais de validação (Pydantic) e padronização de respostas (status_code + payload).
- Sugestões de cenários de teste e casos de borda para cobrir rotas de sucesso e erro.

### Prompts (resumo fiel)
- "Organize um notebook em sessões com CRUD, validações e testes."
- "Sugira casos de borda para validações condicionais e cálculo de dano escalado."
- "Revisar padrão de retorno e erros (códigos HTTP) para endpoints simulados."

### Curadoria e ajustes feitos por mim
- Ajustei o seed para refletir corretamente o domínio (ex.: magia de ataque exige `dano_base`, custo em ouro só quando há componente material).
- Mantive update como PATCH e revalidação do objeto final para evitar estado inválido no fake DB.
